<a href="https://colab.research.google.com/github/RMSCRV/IB2AD/blob/main/Assignment_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from sklearn.datasets import load_diabetes
import pandas as pd
import numpy as np

data = load_diabetes()
data

{'data': array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
          0.01990749, -0.01764613],
        [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
         -0.06833155, -0.09220405],
        [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
          0.00286131, -0.02593034],
        ...,
        [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
         -0.04688253,  0.01549073],
        [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
          0.04452873, -0.02593034],
        [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
         -0.00422151,  0.00306441]]),
 'target': array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
         69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
         68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
         87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
        259.,  53., 190., 142.,  75., 142., 155., 225.,  59

In [ ]:
# Converting data to PyTorch tensors
X = torch.tensor(data.data, dtype=torch.float32)
Y = torch.tensor(data.target, dtype=torch.float32).reshape(-1, 1)       # Reshape Y to be a column vector

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

torch.Size([353, 10]) torch.Size([353, 1])
torch.Size([89, 10]) torch.Size([89, 1])


In [ ]:
# Setting up batches for training
from torch.utils.data import TensorDataset, DataLoader

# Creating TensorDatasets and DataLoaders
train_data = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_data, batch_size=50, shuffle=True)

test_data = TensorDataset(X_test, Y_test)
test_loader = DataLoader(test_data, batch_size=50, shuffle=False)

In [ ]:
import torch
import torch.nn as nn

# Define the model
class DiabetesModel(nn.Module):
  def __init__(self, layer_sizes):
    super(DiabetesModel, self).__init__()
    layers = []
    input_size = layer_sizes[0]
    for output_size in layer_sizes[1:]:
      layers.append(nn.Linear(input_size, output_size))
      if output_size != layer_sizes[-1]:  # Don't add ReLU after the last layer
        layers.append(nn.ReLU())
      input_size = output_size
    self.layers = nn.Sequential(*layers)

  def forward(self, x):
    return self.layers(x)    # pass the data throught the layers

In [ ]:
# Creating a model object, specifying loss (criterion) and optimizer
import torch.optim as optim

model = DiabetesModel(layer_sizes=[X_train.shape[1], 64, 32, 1])
criterion = nn.MSELoss()       # MSE loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

epochs = 1000

for epoch in range(epochs):
  # Use the train_loader to pass inputs (x) and targets (y)
  for inputs, targets in train_loader:
    inputs, targets = inputs.to(device), targets.to(device)

    model.to(device)

    model.train()                         # Put the model object in train model
    optimizer.zero_grad()                 # Reset the gradients
    outputs = model(inputs)               # Create outputs
    loss = criterion(outputs, targets)    # Compare with Y to get loss
    loss.backward                         # Backpropagate the loss
    optimizer.step()                      # Update the parameters based on this round of training

    if (epoch+1) % 10 == 0:
      print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/1000], Loss: 27133.4824
Epoch [10/1000], Loss: 35186.3594
Epoch [10/1000], Loss: 37660.0703
Epoch [10/1000], Loss: 25880.4961
Epoch [10/1000], Loss: 21782.0039
Epoch [10/1000], Loss: 29926.0664
Epoch [10/1000], Loss: 28428.7188
Epoch [10/1000], Loss: 62908.6094
Epoch [20/1000], Loss: 33818.0625
Epoch [20/1000], Loss: 29015.8359
Epoch [20/1000], Loss: 38349.7422
Epoch [20/1000], Loss: 24429.9297
Epoch [20/1000], Loss: 29219.3340
Epoch [20/1000], Loss: 25271.3047
Epoch [20/1000], Loss: 28888.9688
Epoch [20/1000], Loss: 12975.6875
Epoch [30/1000], Loss: 32387.2441
Epoch [30/1000], Loss: 24595.0293
Epoch [30/1000], Loss: 30891.2773
Epoch [30/1000], Loss: 23508.3867
Epoch [30/1000], Loss: 30083.5762
Epoch [30/1000], Loss: 35207.8555
Epoch [30/1000], Loss: 29220.3398
Epoch [30/1000], Loss: 64633.5547
Epoch [40/1000], Loss: 31340.1816
Epoch [40/1000], Loss: 32152.9062
Epoch [40/1000], Loss: 31842.2988
Epoch [40/1000], Loss: 29771.2852
Epoch [40/1000], Loss: 31327.9648
Epoch [40/1000

In [ ]:
# Evaluation
model.eval()
mse_values = []

with torch.no_grad():
  for inputs, targets in test_loader:
    inputs, targets = inputs.to(device), targets.to(device)
    outputs = model(inputs)
    # Calculate the Mean Squared Error
    mse = criterion(outputs, targets)
    mse_values.append(mse.item())

# Average MSE
avg_mse = np.mean(mse_values)
print(f'Average MSE on test set: {avg_mse}')

Average MSE on test set: 26185.552734375


In [ ]:
# Evaluation No.2
model.eval()
predictions = []
actuals = []

with torch.no_grad():
  for inputs, targets in test_loader:
    inputs, targets = inputs.to(device), targets.to(device)
    outputs = model(inputs)
    predictions.extend(outputs.cpu().numpy())
    actuals.extend(targets.cpu().numpy())

results_df = pd.DataFrame({'Predicted': np.array(predictions).flatten(), 'Actual': np.array(actuals).flatten()})
results_df

,Predicted,Actual
0,0.000168,219.0
1,-0.007451,70.0
2,-0.006928,202.0
3,0.008777,230.0
4,-0.000792,111.0
...,...,...
84,-0.007864,153.0
85,-0.012715,98.0
86,-0.016866,37.0
87,-0.011979,63.0


In [ ]:
def train_and_evaluate_model(layer_sizes):
  """
  Trains and evaluates a DiabetesModel with the given layer sizes.

  Args:
    layer_sizes: A list of integers representing the number of neurons in each layer.

  Returns:
    The average Mean Squared Error (MSE) on the test set.
  """
  model = DiabetesModel(layer_sizes=layer_sizes)
  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  epochs = 1000

  for epoch in range(epochs):
    for inputs, targets in train_loader:
      inputs, targets = inputs.to(device), targets.to(device)

      model.train()
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, targets)
      loss.backward()
      optimizer.step()

    if (epoch + 1) % 10 == 0:
      print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

  model.eval()
  mse_values = []

  with torch.no_grad():
    for inputs, targets in test_loader:
      inputs, targets = inputs.to(device), targets.to(device)
      outputs = model(inputs)
      mse = criterion(outputs, targets)
      mse_values.append(mse.item())

  avg_mse = np.mean(mse_values)
  return avg_mse

In [ ]:
architectures = [
    [X_train.shape[1], 32, 1],
    [X_train.shape[1], 64, 32, 1],
    [X_train.shape[1], 128, 64, 32, 1]
]

results = []

for arch in architectures:
    print(f"Training and evaluating model with architecture: {arch}")
    avg_mse = train_and_evaluate_model(arch)
    results.append({'architecture': arch, 'average_mse': avg_mse})
    print(f"Finished architecture {arch} with average MSE: {avg_mse}\n")

display(pd.DataFrame(results))

Training and evaluating model with architecture: [10, 32, 1]
Epoch [10/1000], Loss: 50054.4062
Epoch [20/1000], Loss: 26550.9844
Epoch [30/1000], Loss: 42923.3359
Epoch [40/1000], Loss: 19661.4004
Epoch [50/1000], Loss: 19683.2070
Epoch [60/1000], Loss: 34163.9531
Epoch [70/1000], Loss: 14189.4883
Epoch [80/1000], Loss: 26323.6602
Epoch [90/1000], Loss: 18829.6914
Epoch [100/1000], Loss: 28794.1934
Epoch [110/1000], Loss: 9805.5996
Epoch [120/1000], Loss: 35002.2773
Epoch [130/1000], Loss: 26127.1523
Epoch [140/1000], Loss: 1527.9457
Epoch [150/1000], Loss: 20430.9258
Epoch [160/1000], Loss: 806.5757
Epoch [170/1000], Loss: 12460.8936
Epoch [180/1000], Loss: 11279.6152
Epoch [190/1000], Loss: 28262.2715
Epoch [200/1000], Loss: 9981.4287
Epoch [210/1000], Loss: 10206.9111
Epoch [220/1000], Loss: 13075.4727
Epoch [230/1000], Loss: 7071.0386
Epoch [240/1000], Loss: 1176.0468
Epoch [250/1000], Loss: 676.5114
Epoch [260/1000], Loss: 4975.6943
Epoch [270/1000], Loss: 1365.8851
Epoch [280/100

,architecture,average_mse
0,"[10, 32, 1]",2890.626465
1,"[10, 64, 32, 1]",2829.477783
2,"[10, 128, 64, 32, 1]",2644.785278


In [ ]:
results_df = pd.DataFrame(results)
print(results_df.columns)

Index(['architecture', 'average_mse'], dtype='object')


In [ ]:
# Identify the best performing architecture
best_architecture_index = results_df['average_mse'].idxmin()
best_architecture = results_df.loc[best_architecture_index]

print("Best Performing Architecture:")
print(best_architecture)

# Interpretation of lower MSE
print("\nInterpretation of Lower MSE:")
print("A lower Mean Squared Error (MSE) indicates that the model's predictions are closer to the actual values, on average.")
print("In the context of this regression problem, a lower MSE suggests that the model is better at predicting the progression of diabetes one year after baseline.")

Best Performing Architecture:
architecture    [10, 128, 64, 32, 1]
average_mse              2644.785278
Name: 2, dtype: object

Interpretation of Lower MSE:
A lower Mean Squared Error (MSE) indicates that the model's predictions are closer to the actual values, on average.
In the context of this regression problem, a lower MSE suggests that the model is better at predicting the progression of diabetes one year after baseline.


In [ ]:
# Re-initialize the DiabetesModel with the layer sizes from the best_architecture variable.
model = DiabetesModel(layer_sizes=best_architecture['architecture'])

criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

entire_dataset = TensorDataset(X, Y)
entire_loader = DataLoader(entire_dataset, batch_size=64, shuffle=True)

epochs = 1000

for epoch in range(epochs):
  for inputs, targets in entire_loader:

    inputs, targets = inputs.to(device), targets.to(device)

    model.train()

    optimizer.zero_grad()

    outputs = model(inputs)

    loss = criterion(outputs, targets)

    loss.backward()

    optimizer.step()

  if (epoch + 1) % 100 == 0:
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 3068.3723
Epoch [200/1000], Loss: 3817.3169
Epoch [300/1000], Loss: 2395.4116
Epoch [400/1000], Loss: 2301.6848
Epoch [500/1000], Loss: 2843.2388
Epoch [600/1000], Loss: 2853.8826
Epoch [700/1000], Loss: 2892.9509
Epoch [800/1000], Loss: 1764.9082
Epoch [900/1000], Loss: 2538.9343
Epoch [1000/1000], Loss: 2735.5129


In [ ]:
# Set the trained model to evaluation mode.
model.eval()

predictions = []
actuals = []

with torch.no_grad():
  for inputs, targets in test_loader:

    inputs, targets = inputs.to(device), targets.to(device)

    outputs = model(inputs)

    predictions.extend(outputs.cpu().numpy())
    actuals.extend(targets.cpu().numpy())

results_df = pd.DataFrame({'Predicted': np.array(predictions).flatten(), 'Actual': np.array(actuals).flatten()})

# Calculate the Mean Squared Error (MSE) on the test set.
# Need to convert predictions and actuals back to tensors on the appropriate device for criterion
predictions_tensor = torch.tensor(predictions, dtype=torch.float32).to(device)
actuals_tensor = torch.tensor(actuals, dtype=torch.float32).to(device)
mse = criterion(predictions_tensor, actuals_tensor)

print(f'Mean Squared Error on test set: {mse.item()}')

display(results_df)

Mean Squared Error on test set: 2276.286865234375


,Predicted,Actual
0,143.367905,219.0
1,175.214615,70.0
2,155.368271,202.0
3,304.760529,230.0
4,110.897102,111.0
...,...,...
84,103.586479,153.0
85,75.586090,98.0
86,92.879318,37.0
87,80.921059,63.0


In [ ]:
# Calculate the squared differences between predicted and actual values
results_df['Squared_Difference'] = (results_df['Predicted'] - results_df['Actual'])**2

# Calculate the mean of the squared differences
mean_squared_difference = results_df['Squared_Difference'].mean()

print(f"Manually calculated Mean Squared Error: {mean_squared_difference}")

display(results_df.head())

Manually calculated Mean Squared Error: 2276.286865234375


,Predicted,Actual,Squared_Difference
0,143.367905,219.0,5720.213867
1,175.214615,70.0,11070.115234
2,155.368271,202.0,2174.518066
3,304.760529,230.0,5589.136719
4,110.897102,111.0,0.010588
